In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re


import pickle
from etcpy import etc
from etcpy import tempDep
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [4]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_modified_bestToptNew.csv'),index_col=0)
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,313.238397,10,262,328.717325,7.588735,NaN,-4203.133469,1000,predicted,Mean
P76077,316.667897,10,309,328.717325,7.588735,NaN,-4685.680693,1000,predicted,Mean
P76078,318.596618,10,95,328.717325,7.588735,NaN,-4238.279594,1000,BullShit,Mean
P76079,315.296097,10,248,328.717325,7.588735,NaN,-3013.384455,1000,predicted,Mean
P76081,318.596618,10,356,328.717325,7.588735,NaN,-3714.540347,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,318.180721,10,438,329.166303,4.070000,330.45,-3974.144609,1000,predicted,Exp
P77718,310.911583,10,482,319.125751,4.070000,320.15,-4564.630670,1000,predicted,Exp
P0A6D5,314.624419,10,288,328.762604,7.710000,NaN,-5525.195491,1000,predicted,Mean
standard,317.527053,10,272,328.762604,7.710000,NaN,-3185.073953,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    for react in mae.reactions:
        if match.match(react.id) and react.lower_bound != -1000:
            ori_lb = react.lower_bound
            react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae)}, r2_score(rexp,rae)

In [6]:
S = 0
SMin = 0
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux'])
for i in range(81,101):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm', 'dCpt'])
    rae, score = aerobic(newParams)

    if score > S:
        S = score
        params.to_csv('../data/BestParamsNew.csv')
        bestParams = params
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirstTopt" + str(i) + ".pdf")
        plt.close(fig)
    elif score < SMin:
        SMin = score
        params.to_csv('../data/WorstParamsNew.csv')
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirstTopt" + str(i) + ".pdf")
        plt.close(fig)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'grwoth', 'o2flux', 'gluFlux'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df = pd.concat([df, df1])

81
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.04058868881351919
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.07970111690611638
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.1622858415338603
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.23566104649106687
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.323198396764683
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.4259507278576431
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.5829532720457389
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.8818097605559572
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.1310263732220354
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.3660461136231126
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.391380841955686
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2518103720517395
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is

In [7]:
df.to_csv('../results/modelResultsNew.csv')